In [40]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import DirectoryLoader

import os

# Create data/ragas directory if it doesn't exist
if not os.path.exists("data/ragas"):
    os.makedirs("data/ragas", exist_ok=True)



loader = PyMuPDFLoader("data/PracticalAdviceOnMatrixGames.pdf", mode='single')
docs = loader.load()
# loader = PyMuPDFLoader("data/John Curry, Tim Price - Matrix Games for Modern Wargaming-History of Wargaming Project (2014).pdf", mode='single')
# docs.extend(loader.load())

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [39]:
!curl https://paxsims.wordpress.com/2014/09/21/toward-serious-matrix-games/ -o data/toward-serious-matrix-games.html
!curl https://sites.google.com/view/free-engle-matrix-games/how-to-play-matrix-games -o data/how-to-play-matrix-games.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  137k    0  137k    0     0   229k      0 --:--:-- --:--:-- --:--:--  229k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 82375    0 82375    0     0   419k      0 --:--:-- --:--:-- --:--:--  418k


In [ ]:
loader = DirectoryLoader("data/", glob="*.html")
docs.extend(loader.load())

In [44]:
loader = DirectoryLoader("data/", glob="*.txt")
docs.extend(loader.load())

In [45]:
docs

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-01-01T15:29:59+00:00', 'source': 'data/PracticalAdviceOnMatrixGames.pdf', 'file_path': 'data/PracticalAdviceOnMatrixGames.pdf', 'total_pages': 52, 'format': 'PDF 1.7', 'title': '', 'author': 'Jim Dawson', 'subject': '', 'keywords': '', 'moddate': '2023-01-01T15:29:59+00:00', 'trapped': '', 'modDate': "D:20230101152959+00'00'", 'creationDate': "D:20230101152959+00'00'"}, page_content='Version 15 \nPage 1 of 52  \n© Tom Mouat 2019, 2020, 2022, 2023 \n \n \n \n \nPractical Advice \non \nMatrix Games \nby \nMajor Tom Mouat MBE MSc PGCE \n \n \n \n \n \n \nVersion 15 \n© Tom Mouat 2019, 2020, 2022, 2023\n\x0cVersion 15 \nPage 2 of 52  \n© Tom Mouat 2019, 2020, 2022, 2023 \nContents \nPractical Advice on Matrix Games ..................................................................... 4 \nTerms ......................................................................................

In [2]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

/home/caleb/projects/matrix-game-rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KnowledgeGraph(nodes: 0, relationships: 0)

In [47]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 5, relationships: 0)

In [48]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:  74%|███████▎  | 39/53 [00:04<00:00, 18.64it/s]unable to apply transformation: Invalid json output: The main themes and concepts include voting methods and procedures, consistency and fairness in voting, support and opposition assessment, use of multiple voting options, gradations of support and divisiveness, impact of voting results on social and political dynamics, technology in voting systems, risks and limitations of technological voting, self-voting and competitive behavior among players, arguments and negotiation in game scenarios, relevance of arguments to scenario elements, use of narrative markers and tracking in gameplay, role of imagination, rationality, and common sense in argumentation, managing player interactions and negotiations, introduction of "Turn Zero" for familiarization, and the complexity and variety of arguments in gameplay.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshootin

KnowledgeGraph(nodes: 29, relationships: 55)

In [49]:
kg.save("matrix_games_kg.json")

In [3]:
kg = KnowledgeGraph.load("data/ragas/matrix_games_kg.json")
kg

KnowledgeGraph(nodes: 29, relationships: 55)

In [50]:
from ragas.testset.persona import Persona

persona_new_wargamer = Persona(
    name="New Wargamer",
    role_description="Is new to matrix wargames and needs clear explanations of core concepts, rules, and how to design or participate in a simple game. May ask about the benefits of matrix wargaming.",
)

persona_experienced_designer = Persona(
    name="Experienced Wargame Designer",
    role_description="Has experience designing and running matrix wargames. Interested in advanced design techniques, specific rule nuances, historical examples, and ways to improve game facilitation or player engagement. Might ask for design inspiration for complex scenarios.",
)

persona_academic_researcher = Persona(
    name="Academic Researcher/Analyst",
    role_description="Is using or considering matrix wargames for academic research or professional analysis. Interested in the methodology, validity, limitations, and application of matrix games in specific domains (e.g., political science, military strategy, business). May ask about research papers or case studies.",
)

personas = [persona_new_wargamer, persona_experienced_designer, persona_academic_researcher]

In [58]:
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

In [59]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=kg, persona_list=personas)

In [61]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.5)
]

# query_distribution = [
#         (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
#         (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
#         (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
# ]

In [66]:
from ragas import RunConfig

custom_run_config = RunConfig(max_wait=361, max_retries=20)

testset = generator.generate(testset_size=1000, query_distribution=query_distribution, run_config=custom_run_config, raise_exceptions=False)
pandast_testset = testset.to_pandas()
pandast_testset.to_csv("data/ragas/testset.csv", index=False)

Generating Samples: 100%|██████████| 962/962 [14:14<00:00,  1.13it/s]


In [70]:
pandast_testset.head()

,user_input,reference_contexts,reference,synthesizer_name
0,how matrix games used in cyber operational awa...,[What are Matrix Games? .........................,A Matrix Game used for instruction on the Cybe...,single_hop_specifc_query_synthesizer
1,What role does the Defence Academy of the UK p...,[What are Matrix Games? .........................,The Defence Academy of the UK uses Matrix Game...,single_hop_specifc_query_synthesizer
2,Who or what can be represented by the Actor ca...,[What are Matrix Games? .........................,"In a Matrix Game, ""The USA"" is an example of a...",single_hop_specifc_query_synthesizer
3,how matrix games show clausewitzian friction?,[What are Matrix Games? .........................,"In Matrix Games, ""Actors"" can represent concep...",single_hop_specifc_query_synthesizer
4,"Who or what is ""The Anonymous Hacking Collecti...",[What are Matrix Games? .........................,"""The Anonymous Hacking Collective"" is an examp...",single_hop_specifc_query_synthesizer
